# Brain visualisation

dim of the image is 395*223, there are 1717 images in total

In [ ]:
import h5py
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib.image as mpimg

In [ ]:
f = h5py.File('/content/drive/MyDrive/Brain_data/count_matrix_multisample.h5', 'r')

In [ ]:
list(f.keys())

In [ ]:
dset = f['counts_mat']

In [ ]:
dset.shape

In [ ]:
dset[:,0].shape

In [ ]:
reshaped_to_2d = np.reshape(dset[:,0], (223, -1))
print(reshaped_to_2d.shape)

In [ ]:
def normalize8(I):
  mn = I.min()
  mx = I.max()
  mx -= mn
  I = ((I - mn)/mx) * 255.0 
  return np.round(I).astype(np.uint8)

In [ ]:
normlised = normalize8(reshaped_to_2d)

In [ ]:
img = Image.fromarray(normlised)

In [ ]:
img

In [ ]:
223/4, 395/4

In [ ]:
shrink = normlised[0:220, 0:392]
img_shrink = Image.fromarray(shrink)
img_shrink

In [ ]:
enlarged = np.zeros((223, 395), dtype='uint8')
enlarged[0:220, 0:392] = shrink
img_enlarged = Image.fromarray(enlarged)
img_enlarged

In [ ]:
np.array_equal(enlarged, normlised)

# Create hr png for ESRGAN

In [ ]:
import h5py
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib.image as mpimg
from tqdm import tqdm

In [ ]:
f = h5py.File('/content/drive/MyDrive/Brain_data/count_matrix_multisample.h5', 'r')

In [ ]:
def normalize8(I):
  mn = I.min()
  mx = I.max()
  mx -= mn
  I = ((I - mn)/mx) * 255.0 
  return np.round(I).astype(np.uint8)

In [ ]:
dset = f['counts_mat']
num = dset.shape[1]

shrinks = []
for i in tqdm(range(num)):
  flat_arr = dset[:,i]
  reshaped_to_2d = np.reshape(flat_arr, (223, -1)) 
  shrink = reshaped_to_2d[0:220, 0:392]
  shrinks.append(shrink)

In [ ]:
np.array(shrinks).shape

In [ ]:
shrinks[0].shape

min value in all images are 0, just need to record the maximum value

In [ ]:
max_val = []
savedir = '/content/drive/MyDrive/Brain_data/HR/'
for i in tqdm(range(num)):
  shrink = shrinks[i]
  max = shrink.max()
  max_val.append(max)
  normalised = shrink/max * 255.0 
  arr_8bit = np.round(normalised).astype(np.uint8)
  stacked_img = np.stack((arr_8bit,)*3, axis=-1)
  img = Image.fromarray(stacked_img)
  img.save(savedir + 'brain_peak' + str(i) + '.png')

In [ ]:
import pickle 
with open('/content/drive/MyDrive/Brain_data/max_val.pkl', 'wb') as f:
   pickle.dump(max_val, f)

# Get LR counterpart

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import os

In [ ]:
img = cv2.imread('/content/drive/MyDrive/Brain_data/HR/brain_peak0.png')
img.shape

In [ ]:
img_resized = cv2.resize(img, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_CUBIC)

In [ ]:
img_resized.shape

In [ ]:
cv2_imshow(img_resized)

In [ ]:
cv2.imwrite('test.png', img_resized)

In [ ]:
filelist = os.listdir('/content/drive/MyDrive/Brain_data/HR/')
directory = '/content/drive/MyDrive/Brain_data/HR/'
savedir = '/content/drive/MyDrive/Brain_data/LR/'
for i in tqdm(filelist):
  img = cv2.imread(directory + i)
  img_resized = cv2.resize(img, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_CUBIC)
  cv2.imwrite(savedir + i, img_resized)

In [ ]:
len(os.listdir(savedir))

# Scale back to original size

In [ ]:
import pickle
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
from natsort import natsorted

In [ ]:
with open('/content/drive/MyDrive/Brain_data/scale_factor/max_val.pkl', 'rb') as f:
  max_val = pickle.load(f)

In [ ]:
max_val[0]

In [ ]:
directory = '/content/drive/MyDrive/Brain_data/RES/'
filelist = os.listdir(directory)
filelist.sort()

In [ ]:
sorted = natsorted(filelist)

In [ ]:
enlarged = np.zeros((223, 395), dtype='uint8')
mat = []
for ind, i in tqdm(enumerate(sorted[0:3])):
  img = Image.open(directory + i)
  arr = np.array(img)
  arr_1d = arr[:,:,0]
  base = os.path.splitext(i)[0]
  enlarged[0:220, 0:392] = arr_1d

  max = max_val[ind]
  scaled = enlarged * max / 255.0
  scaled_int = scaled.astype(int)
  mat.append(scaled)

In [ ]:
arr_1d[15:20, 20:30]

In [ ]:
scaled_int = np.matrix.round(scaled)
scaled_int[15:20, 20:30].dtype

In [ ]:
res_scale = np.load('/content/drive/MyDrive/Brain_data/Scaled_RES/brain_peak3.npy')

In [ ]:
plt.imshow(res_scale)

In [ ]:
enlarged = np.zeros((223, 395), dtype='uint8')
savedir = '/content/drive/MyDrive/Brain_data/Scaled_RES/'
for ind, i in tqdm(enumerate(sorted)):
  img = Image.open(directory + i)
  arr = np.array(img)
  arr_1d = arr[:,:,0]
  base = os.path.splitext(i)[0]
  enlarged[0:220, 0:392] = arr_1d

  max = max_val[ind]
  scaled = enlarged * max / 255.0
  np.save(savedir + base + '.npy', scaled)

In [ ]:
len(os.listdir(savedir))

# other 3 Brains visualisation

In [ ]:
import h5py
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib.image as mpimg

In [ ]:
f2 = h5py.File('/content/drive/MyDrive/Brain_data/count_matrix_multisample2.h5', 'r')
f3 = h5py.File('/content/drive/MyDrive/Brain_data/count_matrix_multisample3.h5', 'r')
f4 = h5py.File('/content/drive/MyDrive/Brain_data/count_matrix_multisample4.h5', 'r')

In [ ]:
dset2 = f2['counts_mat']
dset3 = f3['counts_mat']
dset4 = f4['counts_mat']

In [ ]:
print(f2['dim_xy'][1], f2['dim_xy'][0])

In [ ]:
dset2.shape

In [ ]:
print(f3['dim_xy'][1], f3['dim_xy'][0])

In [ ]:
print(f4['dim_xy'][1], f4['dim_xy'][0])

In [ ]:
reshaped_2 = np.reshape(dset2[:,0], (231, -1))
reshaped_3 = np.reshape(dset3[:,0], (167, -1))
reshaped_4 = np.reshape(dset4[:,0], (179, -1))

In [ ]:
def normalize8(I):
  mn = I.min()
  mx = I.max()
  mx -= mn
  I = ((I - mn)/mx) * 255.0 
  return np.round(I).astype(np.uint8)

In [ ]:
normlised_2 = normalize8(reshaped_2)
normlised_3 = normalize8(reshaped_3)
normlised_4 = normalize8(reshaped_4)

In [ ]:
img_2 = Image.fromarray(normlised_2)

In [ ]:
img_2

In [ ]:
img_3 = Image.fromarray(normlised_3)
img_3

In [ ]:
img_4 = Image.fromarray(normlised_4)
img_4

In [ ]:
shrink_2 = normlised_2[0:228, 0:396]
img_shrink_2 = Image.fromarray(shrink_2)
img_shrink_2

In [ ]:
shrink_3 = normlised_3[0:164, 0:424]
img_shrink_3 = Image.fromarray(shrink_3)
img_shrink_3

In [ ]:
shrink_4 = normlised_4[0:176, 0:416]
img_shrink_4 = Image.fromarray(shrink_4)
img_shrink_4

# get HR of the other 3 brains

In [ ]:
import h5py
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib.image as mpimg
from tqdm import tqdm

In [ ]:
f2 = h5py.File('/content/drive/MyDrive/Brain_data/count_matrix_multisample2.h5', 'r')
f3 = h5py.File('/content/drive/MyDrive/Brain_data/count_matrix_multisample3.h5', 'r')
f4 = h5py.File('/content/drive/MyDrive/Brain_data/count_matrix_multisample4.h5', 'r')

In [ ]:
dset2 = f2['counts_mat']
dset3 = f3['counts_mat']
dset4 = f4['counts_mat']

In [ ]:
dset_02 = f2['counts_mat']
num = dset_02.shape[1]

shrinks_02 = []
for i in tqdm(range(num)):
  flat_arr = dset_02[:,i]
  reshaped_to_2d = np.reshape(flat_arr, (231, 399)) 
  shrink_02 = reshaped_to_2d[0:228, 0:396]
  shrinks_02.append(shrink_02)

In [ ]:
dset_03 = f3['counts_mat']
num = dset_03.shape[1]

shrinks_03 = []
for i in tqdm(range(num)):
  flat_arr = dset_03[:,i]
  reshaped_to_2d = np.reshape(flat_arr, (167, -1)) 
  shrink_03 = reshaped_to_2d[0:164, 0:424]
  shrinks_03.append(shrink_03)

In [ ]:
dset_04 = f4['counts_mat']
num = dset_04.shape[1]

shrinks_04 = []
for i in tqdm(range(num)):
  flat_arr = dset_04[:,i]
  reshaped_to_2d = np.reshape(flat_arr, (179, -1)) 
  shrink_04 = reshaped_to_2d[0:176, 0:416]
  shrinks_04.append(shrink_04)

In [ ]:
np.array(shrinks_02).shape

min value in all images are 0, just need to record the maximum value

In [ ]:
max_val_2 = []
savedir = '/content/drive/MyDrive/Brain_data/HR_2/'
for i in tqdm(range(num)):
  shrink = shrinks_02[i]
  max = shrink.max()
  max_val_2.append(max)
  normalised = shrink/max * 255.0 
  arr_8bit = np.round(normalised).astype(np.uint8)
  stacked_img = np.stack((arr_8bit,)*3, axis=-1)
  img = Image.fromarray(stacked_img)
  img.save(savedir + 'brain2peak' + str(i) + '.png')

In [ ]:
max_val_3 = []
savedir = '/content/drive/MyDrive/Brain_data/HR_3/'
for i in tqdm(range(num)):
  shrink = shrinks_03[i]
  max = shrink.max()
  max_val_3.append(max)
  normalised = shrink/max * 255.0 
  arr_8bit = np.round(normalised).astype(np.uint8)
  stacked_img = np.stack((arr_8bit,)*3, axis=-1)
  img = Image.fromarray(stacked_img)
  img.save(savedir + 'brain3peak' + str(i) + '.png')

In [ ]:
max_val_4 = []
savedir = '/content/drive/MyDrive/Brain_data/HR_4/'
for i in tqdm(range(num)):
  shrink = shrinks_04[i]
  max = shrink.max()
  max_val_4.append(max)
  normalised = shrink/max * 255.0 
  arr_8bit = np.round(normalised).astype(np.uint8)
  stacked_img = np.stack((arr_8bit,) * 3, axis=-1)
  img = Image.fromarray(stacked_img)
  img.save(savedir + 'brain4peak' + str(i) + '.png')

In [ ]:
import pickle 
with open('/content/drive/MyDrive/Brain_data/max_val_2.pkl', 'wb') as k:
   pickle.dump(max_val_2, k)

In [ ]:
with open('/content/drive/MyDrive/Brain_data/max_val_3.pkl', 'wb') as j:
   pickle.dump(max_val_3, j)
with open('/content/drive/MyDrive/Brain_data/max_val_4.pkl', 'wb') as q:
   pickle.dump(max_val_4, q)

# Get LR counterpart

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import os

In [ ]:
filelist = os.listdir('/content/drive/MyDrive/Brain_data/HR_2/')
directory = '/content/drive/MyDrive/Brain_data/HR_2/'
savedir = '/content/drive/MyDrive/Brain_data/LR_2/'
for i in tqdm(filelist):
  img = cv2.imread(directory + i)
  img_resized = cv2.resize(img, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_CUBIC)
  cv2.imwrite(savedir + i, img_resized)

In [ ]:
filelist = os.listdir('/content/drive/MyDrive/Brain_data/HR_3/')
directory = '/content/drive/MyDrive/Brain_data/HR_3/'
savedir = '/content/drive/MyDrive/Brain_data/LR_3/'
for i in tqdm(filelist):
  img = cv2.imread(directory + i)
  img_resized = cv2.resize(img, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_CUBIC)
  cv2.imwrite(savedir + i, img_resized)

In [ ]:
filelist = os.listdir('/content/drive/MyDrive/Brain_data/HR_4/')
directory = '/content/drive/MyDrive/Brain_data/HR_4/'
savedir = '/content/drive/MyDrive/Brain_data/LR_4/'
for i in tqdm(filelist):
  img = cv2.imread(directory + i)
  img_resized = cv2.resize(img, None, fx=0.25, fy=0.25, interpolation=cv2.INTER_CUBIC)
  cv2.imwrite(savedir + i, img_resized)

In [ ]:
len(os.listdir(savedir))

# Scale back to original size

In [ ]:
import pickle
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

In [ ]:
from natsort import natsorted

## Brain 2

In [ ]:
with open('/content/drive/MyDrive/Brain_data/scale_factor/max_val_2.pkl', 'rb') as f:
  max_val_2 = pickle.load(f)

In [ ]:
directory = '/content/drive/MyDrive/Brain_data/RES_2/'
filelist = os.listdir(directory)
filelist.sort()

In [ ]:
sorted = natsorted(filelist)

In [ ]:
enlarged = np.zeros((231, 399), dtype='uint8')
savedir = '/content/drive/MyDrive/Brain_data/Scaled_RES_2/'
for ind, i in tqdm(enumerate(sorted)):
  img = Image.open(directory + i)
  arr = np.array(img)
  arr_1d = arr[:,:,0]
  base = os.path.splitext(i)[0]
  enlarged[0:228, 0:396] = arr_1d

  max = max_val_2[ind]
  scaled = enlarged * max / 255.0
  np.save(savedir + base + '.npy', scaled)

In [ ]:
len(os.listdir(savedir))

## Brain 3

In [ ]:
with open('/content/drive/MyDrive/Brain_data/scale_factor/max_val_3.pkl', 'rb') as f:
  max_val_3 = pickle.load(f)

In [ ]:
directory = '/content/drive/MyDrive/Brain_data/RES_3/'
filelist = os.listdir(directory)
filelist.sort()

In [ ]:
sorted = natsorted(filelist)

In [ ]:
enlarged = np.zeros((167, 424), dtype='uint8')
savedir = '/content/drive/MyDrive/Brain_data/Scaled_RES_3/'
for ind, i in tqdm(enumerate(sorted)):
  img = Image.open(directory + i)
  arr = np.array(img)
  arr_1d = arr[:,:,0]
  base = os.path.splitext(i)[0]
  enlarged[0:164, 0:424] = arr_1d

  max = max_val_3[ind]
  scaled = enlarged * max / 255.0
  np.save(savedir + base + '.npy', scaled)

In [ ]:
len(os.listdir(savedir))

## Brain 4

In [ ]:
with open('/content/drive/MyDrive/Brain_data/scale_factor/max_val_4.pkl', 'rb') as f:
  max_val_4 = pickle.load(f)

In [ ]:
directory = '/content/drive/MyDrive/Brain_data/RES_4/'
filelist = os.listdir(directory)
filelist.sort()

In [ ]:
sorted = natsorted(filelist)

In [ ]:
sorted[0]

In [ ]:
enlarged = np.zeros((179, 419), dtype='uint8')
savedir = '/content/drive/MyDrive/Brain_data/Scaled_RES_4/'
for ind, i in tqdm(enumerate(sorted)):
  img = Image.open(directory + i)
  arr = np.array(img)
  arr_1d = arr[:,:,0]
  base = os.path.splitext(i)[0]
  enlarged[0:176, 0:416] = arr_1d

  max = max_val_4[ind]
  scaled = enlarged * max / 255.0
  np.save(savedir + base + '.npy', scaled)

In [ ]:
len(os.listdir(savedir))